# Nonlinear Solve

From [their documentation](https://docs.sciml.ai/NonlinearSolve/stable/):\
NonlinearSolve.jl is a unified interface for the nonlinear solving packages of Julia. The package includes its own high-performance nonlinear solvers which include the ability to swap out to fast direct and iterative linear solvers, along with the ability to use sparse automatic differentiation for Jacobian construction and Jacobian-vector products.

In [ ]:
using Pkg
Pkg.add("NonlinearSolve")
using NonlinearSolve

Here's the simplest example from their documentation:

In [75]:
f(u, p) = u .* u .- p # Our nonlinear function 
u0 = [1.0, 1.0] # initial guess 
p = 2 # Extra parameter
prob = NonlinearProblem(f, u0, p) # Package it into the NonlinearProblem object 
sol = solve(prob, NewtonRaphson(); store_trace=Val(true)) # Run the solver. In this case, we're using Newton-Raphson to solve, but we could choose other methods.

retcode: Success
u: 2-element Vector{Float64}:
 1.4142135623730951
 1.4142135623730951

### A more verbose explanation of what happened
We define the vector-valued function as follows:\
$$f(u, p) = \begin{bmatrix} u_1^2 - p \\ u_2^2 - p \end{bmatrix}$$
where $u = [u_1, u_2]$ is the vector of unknowns, and $p$ is a parameter (in this case, a scalar)\
In our case, $p = 2$. So we want to solve:$$f(u, 2) = \begin{bmatrix} u_1^2 - 2 \\ u_2^2 - 2 \end{bmatrix} = \begin{bmatrix} 0 \\ 0 \end{bmatrix}$$
To start, we defined an initial guess $u_0 = [1.0, 1.0]$.\
Then, we defined a `NonlinearProblem` object with our function, initial guess, and extra parameter.\
We then passed this `NonlinearProblem` alongside our chosen approximation method to the [`solve` function](https://docs.sciml.ai/NonlinearSolve/stable/basics/solve/) to solve the problem.\
We chose Newton-Raphson as our solving method, but there are [other methods](https://docs.sciml.ai/NonlinearSolve/stable/native/solvers/) that we could have used.

The solver correctly found $\pm \sqrt2$ as the solutions to $u_1,u_2$. We can check how many iterations it took by checking the .stats field of our solution object:

In [73]:
sol.stats

SciMLBase.NLStats
Number of function evaluations:                    7
Number of Jacobians created:                       6
Number of factorizations:                          0
Number of linear solves:                           5
Number of nonlinear solver iterations:             5

We can also see the trace of the solution:

In [76]:
sol.trace

----    	-------------       	-----------         
Iter    	f(u) inf-norm       	Step 2-norm         
----    	-------------       	-----------         
0       	1.00000000e+00      	0.00000000e+00      
1       	2.50000000e-01      	7.07106781e-01      
2       	6.94444444e-03      	1.17851130e-01      
3       	6.00730488e-06      	3.46620971e-03      
4       	4.51061410e-12      	3.00364793e-06      
5       	4.44089210e-16      	2.25530705e-12      


The `inf-norm` tells us how far away our largest evaluation is from $0$ \
For example, it starts off at $1$ because our initial guess $u = [1.0, 1.0]$ makes our equations evaluate to $1^2 - 2 = -1, |-1| = 1$
The `Step 2-norm` tells us how much our guess has moved since the last iteration. The smaller, the closer we are.

### Another, more complex example

In [77]:
f(u, p) = [ # Nonlinear function with variables that interact with one another
    u[1]^2 + u[2]^2 - p[1],
    u[1] - exp(u[2]) + p[2]
] 
u0 = [1.0, 1.0] # Initial guess
p = [1.0, 2.0] # Extra parameters
prob = NonlinearProblem(f, u0, p)
sol = solve(prob, NewtonRaphson(); store_trace=Val(true))

retcode: Success
u: 2-element Vector{Float64}:
 0.44663791193705465
 0.8947148012749693

In [78]:
sol.stats

SciMLBase.NLStats
Number of function evaluations:                    7
Number of Jacobians created:                       6
Number of factorizations:                          0
Number of linear solves:                           5
Number of nonlinear solver iterations:             5

In [79]:
sol.trace

----    	-------------       	-----------         
Iter    	f(u) inf-norm       	Step 2-norm         
----    	-------------       	-----------         
0       	1.00000000e+00      	0.00000000e+00      
1       	1.98187535e-01      	4.45182586e-01      
2       	1.29015848e-02      	1.13585143e-01      
3       	6.06576748e-05      	7.78830372e-03      
4       	1.24416299e-09      	3.52726915e-05      
5       	0.00000000e+00      	7.00045662e-10      


Here, the special thing to note is that `p` can also be a vector of values. 

## Question
What precisely are the real-world applications of NonlinearSolve?
## Project idea
It's a very general package that does a lot, so there are a lot of projects that could be done. A simulation of the [Lotka-Volterra equations](https://en.wikipedia.org/wiki/Lotka%E2%80%93Volterra_equations) would be interesting; solving for the "steady states" of a predator & prey population, changing their "predator" or "prey" values to see how a population might grow or shrink depending on how aggressive/passive a creature is.